In [3]:
from tokenizers import Tokenizer
from tokenizers.models import BPE
from tokenizers.trainers import BpeTrainer

In [4]:
unk_token = "<UNK>"
spl_tokens = ["<UNK>", "<SEP>", "<MASK>", "<CLS>"]
tokenizer = Tokenizer(BPE(unk_token = unk_token))
trainer = BpeTrainer(vocab_size=10000, min_frequency=2, special_tokens=spl_tokens)

In [ ]:
tokenizer.train(['description'], trainer)
tokenizer.save("../data/tokenizer-trained.json")
tokenizer = Tokenizer.from_file("../data/tokenizer-trained.json")

In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import pickle

# import spacy
# import ru_core_news_lg

from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline

from sklearn.model_selection import train_test_split

from sklearn.metrics import roc_auc_score

In [2]:
# # python -m spacy download ru_core_news_lg

# # nlp = spacy.load("ru_core_news_sm")
# nlp = spacy.load("ru_core_news_lg")

# [' '.join([tok.lemma_ for tok in doc]) 
#  for doc in 
#  nlp.pipe(val['description'], batch_size=512, n_process=30, disable=["parser", "ner"])
# ]

In [4]:
val = pd.read_csv('../data/lemmatized_val.csv')
val.head()

,title,description,subcategory,category,price,region,city,datetime_submitted,is_bad,lemmatized_desc
0,Шины,Звонить 89425546881,Запчасти и аксессуары,Транспорт,2000.0,Тульская область,Огаревка,2019-10-10 00:00:25.200714,1,звонить 89425546881
1,Продается мобильная перегородка с дверью,"Мобильная перегородка, предназначена для разгр...",Оборудование для бизнеса,Для бизнеса,10500.0,Вологодская область,Вологда,2019-10-10 00:03:11.527292,0,"мобильный перегородка , предназначить для разг..."
2,Комплект зимних шин на дисках 682/32/64,Шины зимние б/у Marshal Assimetric I”Zen KW 61...,Запчасти и аксессуары,Транспорт,4000.0,Россия,Москва,2019-10-10 00:05:07.193248,1,шина зимний б/у marshal assimetric i”zen kw 61...
3,Кровать-трансормер Дакота сб-4085,"Продаю кровать-трансформер производства ""Столп...",Мебель и интерьер,Для дома и дачи,17000.0,Московская область,Химки,2019-10-10 00:05:58.165179,0,"продавать кровать - трансформер производство ""..."
4,Honda VFR 800 2004 г.в,"Мот в отличном состоянии для своих лет, Родной...",Мотоциклы и мототехника,Транспорт,235000.0,Брянская область,Брянск,2019-10-10 00:06:19.231151,0,"мот в отличный состояние для свой год , родной..."


In [5]:
val['at_sign'] = val['description'].str.contains('@')
val['email'] = val['description'].str.contains('[a-zA-Z\d\._\-]*@[a-zA-Z\d]*\.?')
val['telegram'] = val['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]')
val['telegram_2'] = val['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]\s*(:|-)?\s*[a-zA-Z\d]')
val['telegram_3'] = val['description'].str.contains('[Tt][gG]\s*(:|-)?\s*[a-zA-Z\d]')
val['num_1'] = val['description'].str.contains('\+[\s]*7')
val['num_2'] = val['description'].str.contains('8[-\s]*9')
val['num_3'] = val['description'].str.contains('8[-\s]*\(\s*9')
val['vk_dot_com'] = val['description'].str.contains('[vV][kK]\.com')
val['vk_dot_com_2'] = val['description'].str.contains('[vV][kK]\.com/[a-zA-Z\d]')
val['instagram'] = val['description'].str.contains('[Ii][nN][sS][tT]')
val['instagram_2'] = val['description'].str.contains('[Ii][nN][sS][tT]\s*(:|-)')
val['instagram_3'] = val['description'].str.contains('[Ii][nN][sS][tT]\s*(:|-)\s*[a-zA-Z\d]')

/Users/rkhairullin/anaconda3/lib/python3.7/site-packages/pandas/core/strings.py:1954: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)


In [4]:
train = pd.read_csv('../data/lemmatized_train.csv')

NameError: name 'pd' is not defined

In [7]:
train['at_sign'] = train['description'].str.contains('@')
train['email'] = train['description'].str.contains('[a-zA-Z\d\._\-]*@[a-zA-Z\d]*\.?')
train['telegram'] = train['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]')
train['telegram_2'] = train['description'].str.contains('[Tt][eE][lL][eE][gG][Rr][Aa][mM]\s*(:|-)?\s*[a-zA-Z\d]')
train['telegram_3'] = train['description'].str.contains('[Tt][gG]\s*(:|-)?\s*[a-zA-Z\d]')
train['num_1'] = train['description'].str.contains('\+[\s]*7')
train['num_2'] = train['description'].str.contains('8[-\s]*9')
train['num_3'] = train['description'].str.contains('8[-\s]*\(\s*9')
train['vk_dot_com'] = train['description'].str.contains('[vV][kK]\.com')
train['vk_dot_com_2'] = train['description'].str.contains('[vV][kK]\.com/[a-zA-Z\d]')
train['instagram'] = train['description'].str.contains('[Ii][nN][sS][tT]')
train['instagram_2'] = train['description'].str.contains('[Ii][nN][sS][tT]\s*(:|-)')
train['instagram_3'] = train['description'].str.contains('[Ii][nN][sS][tT]\s*(:|-)\s*[a-zA-Z\d]')

In [8]:
val[['category', 'is_bad']].groupby(by=['category']).agg(['count', 'sum'])

is_bad      
                     count   sum
category                        
Бытовая электроника   2788   160
Для бизнеса            289    23
Для дома и дачи       1918   342
Животные               210   105
Личные вещи           1765   187
Недвижимость          1757   835
Работа                 544   270
Транспорт             5497  1690
Услуги                 672   311
Хобби и отдых          797    58

In [9]:
train[['category', 'is_bad']].groupby(by=['category']).agg(['count', 'sum'])

is_bad       
                      count    sum
category                          
Бытовая электроника  152132  21010
Для бизнеса           11958   1749
Для дома и дачи      132120  30087
Животные              31421  17159
Личные вещи          199655  27509
Недвижимость         100796  36694
Работа                26208  10340
Транспорт            209221  54813
Услуги                56681  28430
Хобби и отдых         64295  10475

In [374]:
def build_model(category, features=['lemmatized_desc', 'category', 'at_sign', 'email', 'telegram', 'telegram_2', 'telegram_3',
                                    'num_1', 'num_2', 'num_3', 'vk_dot_com', 'vk_dot_com_2', 'instagram', 'instagram_2', 'instagram_3']):
    pipeline = Pipeline([
        ('transformer', 
         ColumnTransformer([
             ('tf-idf', 
              TfidfVectorizer(
                  max_df = 0.2,
                  min_df = 0.0005,
                  strip_accents='unicode',
                  ngram_range = (1, 2),
                  lowercase=True
              ), 'lemmatized_desc'),
             ('OHE',
              OneHotEncoder(
                  handle_unknown='ignore'
              ), ['category'])
         ])),
        ('classifier', 
         LogisticRegression(
             solver='saga',
             penalty='l1', 
             random_state=42,
             max_iter=1000,
             verbose=1,
             tol=0.00005,
             n_jobs=-1)
        )
    ])
    pipeline.fit(train[features][train['category'] == category], train['is_bad'][train['category'] == category])
    return pipeline


def eval_model(model, category, features=['lemmatized_desc', 'category', 'at_sign', 'email', 'telegram', 'telegram_2', 'telegram_3',
                                          'num_1', 'num_2', 'num_3', 'vk_dot_com', 'vk_dot_com_2', 'instagram', 'instagram_2', 'instagram_3']):
    y_pred = pipeline.predict_proba(val[features][val['category'] == category])[:, 1]
    clipped_roc_auc = roc_auc_score(val['is_bad'][val['category'] == category], np.clip(0, 1, y_pred + val['num_1'][val['category'] == category]))
    normal_roc_auc = roc_auc_score(val['is_bad'][val['category'] == category], y_pred)
    return clipped_roc_auc, normal_roc_auc

In [375]:
categories = train.category.unique()
models = [build_model(category) for category in categories]

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 807 epochs took 1243 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 20.7min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 2832 seconds


/Users/rkhairullin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 47.2min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


...

[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:  1.1min finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 34 seconds


/Users/rkhairullin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   34.3s finished
[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


convergence after 231 epochs took 29 seconds


[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed:   29.5s finished


In [376]:
scores = [eval_model(model, category) for model, category in zip(models, categories)]

In [377]:
scores

[(0.8030230504348859, 0.8016770193249622),
 (0.7543094859516026, 0.7378645379190933),
 (0.745735764355889, 0.7408992527010537),
 (0.8569896308980213, 0.840767694063927),
 (0.7866875087490085, 0.7836778498436844),
 (0.8105755610228882, 0.7993229092535736),
 (0.7390838713029473, 0.698223076623326),
 (0.7600907029478459, 0.756734693877551),
 (0.9065871199738477, 0.9221150702844066),
 (0.6953906461205732, 0.6918356312516897)]

In [371]:
scores

[(0.8030230504348859, 0.8016770193249622),
 (0.7543094859516026, 0.7378645379190933),
 (0.745735764355889, 0.7408992527010537),
 (0.8569896308980213, 0.840767694063927),
 (0.7866875087490085, 0.7836778498436844),
 (0.8105755610228882, 0.7993229092535736),
 (0.7390838713029473, 0.698223076623326),
 (0.7600907029478459, 0.756734693877551),
 (0.9065871199738477, 0.9221150702844066),
 (0.6953906461205732, 0.6918356312516897)]

In [380]:
print('\n'.join(categories))

Для дома и дачи
Транспорт
Услуги
Бытовая электроника
Хобби и отдых
Личные вещи
Недвижимость
Животные
Для бизнеса
Работа


In [10]:
pipeline = Pipeline([
    ('transformer', 
     ColumnTransformer([
         ('tf-idf', 
          TfidfVectorizer(
              max_df = 0.2,
              min_df = 0.0005,
              strip_accents='unicode',
              ngram_range = (1, 2),
              lowercase=True
          ), 'lemmatized_desc'),
         ('OHE',
          OneHotEncoder(
              handle_unknown='ignore'
          ), ['category'])
     ])),
    ('classifier', 
     LogisticRegression(
         solver='saga',
         penalty='l1', 
         random_state=42,
         verbose=1,
         tol=0.00005,
         n_jobs=-1)
    )
])

In [11]:
pipeline.fit(train[['lemmatized_desc', 'category', 'at_sign', 'email', 'telegram', 'telegram_2', 'telegram_3', 'num_1', 'num_2', 'num_3', 
                    'vk_dot_com', 'vk_dot_com_2', 'instagram', 'instagram_2', 'instagram_3']
                  ][train['category'] != 'Для бизнеса'], 
             train['is_bad'][train['category'] != 'Для бизнеса'])

[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 4 concurrent workers.


max_iter reached after 3039 seconds


/Users/rkhairullin/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_sag.py:354: ConvergenceWarning: The max_iter was reached which means the coef_ did not converge
  ConvergenceWarning,
[Parallel(n_jobs=-1)]: Done   1 out of   1 | elapsed: 50.7min finished


Pipeline(steps=[('transformer',
                 ColumnTransformer(transformers=[('tf-idf',
                                                  TfidfVectorizer(max_df=0.2,
                                                                  min_df=0.0005,
                                                                  ngram_range=(1,
                                                                               2),
                                                                  strip_accents='unicode'),
                                                  'lemmatized_desc'),
                                                 ('OHE',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['category'])])),
                ('classifier',
                 LogisticRegression(n_jobs=-1, penalty='l1', random_state=42,
                                    solver='saga', tol=5e-05, verbose=1))])

In [28]:
y_pred = pipeline.predict_proba(val[['lemmatized_desc', 'category', 'at_sign', 'email', 'telegram', 'telegram_2', 'telegram_3', 'num_1', 'num_2', 'num_3', 
                                     'vk_dot_com', 'vk_dot_com_2', 'instagram', 'instagram_2', 'instagram_3']])

In [13]:
roc_auc_score(val['is_bad'][val['category'] != 'Для бизнеса'], np.clip(0, 1, y_pred[:, 1] + val['num_1'][val['category'] != 'Для бизнеса']))

0.9323972394040682

In [14]:
roc_auc_score(val['is_bad'][val['category'] != 'Для бизнеса'], y_pred[:, 1])

0.9312136060832232

## some analysis

In [351]:
pipeline['classifier'].coef_[0, -14:]

array([-0.77611639,  0.        ,  0.07645648,  0.55592762, -1.28061452,
       -0.93750908, -0.63158339,  0.        , -0.98941384,  0.22220631,
        1.18121115,  0.0991847 ,  1.00454921, -1.32709629])

In [275]:
for i, elem in test[
    (abs(test['is_bad'] - y_pred[:, 1]) > 0.5) &
    (test['category'] == 'Для бизнеса')
][:10].iterrows():
    print(i, '\n', elem['description'], '\n', elem['is_bad'], '\n', y_pred[:, 1][i], '\n\n\n\n')

213 
 Э/двигатель МТКН511-2 33/920 IM1006 
 0 
 0.6548410064245151 




756 
 Торговое Оборудование - В КРЕДИТ с минимальными ставками!/
 /
Маникюрный стол М-59/
 /
---/
 /
5500 х 200 х 720/
 /
____________________________________________________ proMEBELnet Собственное производство торгового оборудования и не только:/
 /
Мы предоставим ВАМ: /
 /
✅ Консультация по договору аренды и выбора торговой площади/
 /
✅ Технический дизайн-проект 6Д и технически правильную визуализацию оборудования/
 /
✅ Изготовление любого торгового оборудования на заказ в кратчайшие сроки за счет собственной производственной базы/
 /
✅ Продажа-монтаж-обслуживание: систем видеонаблюдения и GSM охраны/
 /
С НАМИ возможно:/
 /
🔴 Удаленное согласование WhatsApp📲 /
 /
🔴 Помощь в открытие нового бизнеса /
 /
🔴 Современный подход к согласованию и реализации ваших идей /
 /
💥Каждый наш товар это индивидуальный клиент с индивидуальным заказом💥/
 /
📌ПИШИТЕ📲ЗВОНИТЕ🚘ЗАЕЗЖАЙТЕ К НАМ 
 1 
 0.10968137478436124 




761 
 Вит

In [276]:
(pipeline['transformer'].transform(test[
    ['lemmatized_desc', 'category', 'telegram', 'telegram_2', 'telegram_3', 'num_1', 'num_2', 'num_3', 
     'vk_dot_com', 'vk_dot_com_2', 'instagram', 'instagram_2', 'instagram_3']
].iloc[7524:7525]).toarray()).shape

(1, 18770)

In [278]:
shit = [i for i, elem in enumerate(pipeline['transformer'].transform(test[
    ['lemmatized_desc', 'category', 'telegram', 'num_1', 'num_2', 'vk_dot_com', 'instagram']].iloc[7524:7525]).toarray()[0]) if elem != 0]

In [300]:
sum(data['lemmatized_desc'].str.contains('сдаваться'))

10474

In [279]:
print('\n'.join(np.array(pipeline['transformer'].get_feature_names())[shit]))

tf-idf__79
tf-idf__автомоика
tf-idf__автосервис
tf-idf__аренда
tf-idf__аренда помещение
tf-idf__весь
tf-idf__весь вопрос
tf-idf__вид
tf-idf__вид деятельность
tf-idf__вопрос
tf-idf__вопрос по
tf-idf__город
tf-idf__деиствовать
tf-idf__деятельность
tf-idf__любои
tf-idf__любои вид
tf-idf__можно
tf-idf__можно под
tf-idf__по телефон
tf-idf__под
tf-idf__под любои
tf-idf__помещение
tf-idf__пять
tf-idf__сдаваться
tf-idf__сдаваться аренда
tf-idf__телефон
tf-idf__территория
tf-idf__центр
tf-idf__центр город
OHE__x0_Для бизнеса


/Users/rkhairullin/anaconda3/lib/python3.7/site-packages/sklearn/utils/deprecation.py:87: FutureWarning: Function get_feature_names is deprecated; get_feature_names is deprecated in 1.0 and will be removed in 1.2. Please use get_feature_names_out instead.
  warnings.warn(msg, category=FutureWarning)


In [221]:
print(test.iloc[1510]['lemmatized_desc'])

покрыть порошковый краска металлический элемент , оцинкованный крепеж/ 
  / 
 наш компания ооо " эльбрус реал эстэйт " заниматься продажа и установка оборудование для детский и спортивный площадка в сочи и краснодарский край , а также укладка резиновый покрытие из epdm крошка на детский и спортивный площадках./ 
  / 
 мы являться официальный дилер завод , что давать нам возможность предложить минимальный цена на оборудование . также осуществлять полный комплекс работа под ключ от проектирование площадка и подготовка основание , до поставка и установка оборудования:/ 
  / 
 необходимый сделать площадка , но не знать как она быть выглядеть ? Для каждый клиент мы предлагать бесплатный услуга 5-d визуализация будущий площадка с выбрать оборудованием./ 
  / 
 оставить заявка на сайт или по телефон +7 ( 843 ) 336 - 75 - 16 и мы подобрать вам оборудование , сделать бесплатный 5-d визуализация площадка и рассчитать стоимость площадка под ключ .


In [280]:
test.iloc[7524]

title                                                Автомойка в аренду
description           сдается в аренду помещение в центре города...д...
subcategory                                              Готовый бизнес
category                                                    Для бизнеса
price                                                             50000
region                                               Московская область
city                                                           Серпухов
datetime_submitted                           2019-10-12 10:37:05.039184
is_bad                                                                1
lemmatized_desc       сдаваться в аренда помещение в центр город ......
telegram                                                          False
num_1                                                             False
num_2                                                              True
vk_dot_com                                                      

In [187]:
test.iloc[45]

title                                                        Подработка
description           ✅Открылся новая группа для подработки, для раз...
subcategory                                           Предложение услуг
category                                                         Услуги
price                                                                 0
region                                                Орловская область
city                                                               Орёл
datetime_submitted                           2019-10-10 00:55:38.436904
is_bad                                                                1
lemmatized_desc       ✅ открыться новый группа для подработка , для ...
telegram                                                          False
num_1                                                             False
num_2                                                             False
vk_dot_com                                                      

## some analysis end

In [324]:
scores = []
for category in test['category'].unique():
    scores.append(
        roc_auc_score(
            test['is_bad'][test['category'] == category], 
            np.clip(0, 1, y_pred[test['category'] == category][:, 1] + test[test['category'] == category]['num_1'])
        )
    )
np.mean(scores)

0.8686439753710262

In [29]:
scores = []
for category in val['category'].unique():
    if category != 'Для бизнеса':
        scores.append(
            roc_auc_score(
                val['is_bad'][val['category'] == category], 
                np.clip(0, 1, y_pred[val['category'] == category][:, 1] + val[val['category'] == category]['num_1'])
            )
        )
np.mean(scores)

0.8752085538521832

In [30]:
scores

[0.9705863381531685,
 0.8850725057143706,
 0.8349718387182042,
 0.8380035806218882,
 0.9353809931506849,
 0.8895294010677127,
 0.8394381970043395,
 0.8360029737766964,
 0.847891156462585]

In [356]:
scores

[0.9709719560510613,
 0.7968290290944754,
 0.884176388517826,
 0.8350396155696983,
 0.8497252184446562,
 0.9349671803652968,
 0.889021523114292,
 0.8383183239232886,
 0.8378277912949444,
 0.8482539682539683]

In [274]:
print('\n'.join(test['category'].unique()))

Транспорт
Для бизнеса
Для дома и дачи
Личные вещи
Услуги
Бытовая электроника
Недвижимость
Хобби и отдых
Работа
Животные


In [326]:
scores = []
for category in test['category'].unique():
    scores.append(
        roc_auc_score(
            test['is_bad'][test['category'] == category], 
            y_pred[test['category'] == category][:, 1]
        )
    )
scores

[0.9704624617063241,
 0.8115397188623734,
 0.8880308056520319,
 0.7918437336911952,
 0.8549358249236223,
 0.9326674277016742,
 0.8828074869783211,
 0.8310624795856469,
 0.8372871046228709,
 0.8459863945578231]

In [329]:
np.mean(scores)

0.8646623438281884

In [327]:
print('\n'.join(test.category.unique()))

Транспорт
Для бизнеса
Для дома и дачи
Личные вещи
Услуги
Бытовая электроника
Недвижимость
Хобби и отдых
Работа
Животные


In [99]:
y_pred_train = pipeline.predict_proba(data[['lemmatized_desc', 'price', 'category', 'telegram', 'num_1', 'num_2', 'vk_dot_com', 'instagram']].iloc[:5])

In [ ]:
scores_train = []
for category in data['category'].unique():
    scores_train.append(
        roc_auc_score(
            data['is_bad'][data['category'] == category], 
            y_pred_train[data['category'] == category][:, 1]
        )
    )

In [ ]:
np.mean(scores_train)

In [ ]:
scores_train

In [ ]:
print('\n'.join(data.category.unique()))